In [ ]:
import customtkinter as ctk
from PIL import Image, ImageTk
import datetime
import pyodbc as odbc
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import functools
from tkcalendar import DateEntry

conn = odbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=SAMEER\SQLEXPRESS;"
    r"DATABASE=version1;"
    r"Trusted_Connection=yes;"
)
    
cursor = conn.cursor()

# Setup
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")

app = ctk.CTk()
app.title("ElectroSales - Dashboard")
app.geometry("1350x700+0+0")

import datetime

# Get current date and time
now = datetime.datetime.now()
formatted_datetime = now.strftime("%A, %d %B %Y | %I:%M %p")

open_windows = {}

# Sidebar
sidebar = ctk.CTkFrame(app, width=200, corner_radius=0, fg_color="#1E2A38")
sidebar.pack(side="left", fill="y")

ctk.CTkLabel(sidebar, text="Menu", font=("Arial", 18, "bold"), text_color="white").pack(pady=(20, 10))

menu_items = ["Dashboard", "Inventory", "Billing", "Quotation", "Invoices", "Order status"]
for item in menu_items:
    btn_color = "#0F62FE" if item == "Dashboard" else "#2C3E50"
    btn = ctk.CTkButton(sidebar, text=item, width=180, fg_color=btn_color, corner_radius=8)
    btn.pack(pady=5)

# Top Bar
topbar = ctk.CTkFrame(app, height=50, fg_color="#141C2B")
topbar.pack(fill="x", side="top")

search = ctk.CTkEntry(topbar, placeholder_text="Search products, customers, invoices...", width=600)
search.pack(padx=20, pady=10, side="left")

# Profile (right side)
ctk.CTkLabel(topbar, text="🧑 John Smith\nAdministrator", font=("Arial", 10), anchor="e").pack(side="right", padx=20)

# Main Content
main_frame = ctk.CTkFrame(app, fg_color="#0F172A")
main_frame.pack(fill="both", expand=True, padx=15, pady=10)

top_bar = ctk.CTkFrame(main_frame, fg_color="#1E293B")
top_bar.pack(fill="x", pady=(10, 5), padx=20)

welcome_label = ctk.CTkLabel(top_bar, text="👋 Welcome Admin", font=("Arial", 20, "bold"), text_color="#F8FAFC")
welcome_label.pack(side="left", padx=20)

datetime_label = ctk.CTkLabel(top_bar, font=("Arial", 16), text_color="#CBD5E1")
datetime_label.pack(side="right", padx=20)

def update_time():
    now = datetime.datetime.now()
    formatted = now.strftime("%A, %d %B %Y | %I:%M %p")
    datetime_label.configure(text=formatted)
    app.after(60000, update_time)  # update every 60,000 ms = 1 min

update_time()

# Cards (Revenue, Products, Customers, Orders)
info_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
info_frame.pack(fill="x", pady=(10, 0))

def open_revenue_window():
    app.lower()
    revenue_window = ctk.CTkToplevel(app)
    revenue_window.title("Total Revenue")
    revenue_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    revenue_window.state("zoomed")
    revenue_window.configure(fg_color="#0F172A")

    revenue_window.lift()
    revenue_window.attributes("-topmost", True)
    revenue_window.focus_force()
    revenue_window.after(500, lambda: revenue_window.attributes("-topmost", False))
    revenue_window.protocol("WM_DELETE_WINDOW", lambda: [revenue_window.destroy(), app.lift()])

    ctk.CTkLabel(revenue_window, text="💰 Total Revenue", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(revenue_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [revenue_window.destroy(), app.lift()]).pack(pady=10)

def open_products_window():
    app.lower()
    products_window = ctk.CTkToplevel(app)
    products_window.title("Total Products")
    products_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    products_window.state("zoomed")
    products_window.configure(fg_color="#0F172A")

    products_window.lift()
    products_window.attributes("-topmost", True)
    products_window.focus_force()
    products_window.after(500, lambda: products_window.attributes("-topmost", False))
    products_window.protocol("WM_DELETE_WINDOW", lambda: [products_window.destroy(), app.lift()])

    ctk.CTkLabel(products_window, text="📦 Total Products", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(products_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [products_window.destroy(), app.lift()]).pack(pady=10)

def fetch_employees():
    cursor.execute("""
        SELECT e.ID, e.Name, e.Designation, e.BasicSalary, e.PerDaySalary, e.Status,
               e.JoiningDate, e.ResigningDate, 
               s.OTDayAmount, s.OTHours, s.AdvanceTaken, s.AdvanceDeducted,
               (e.BasicSalary + (s.OTDayAmount * s.OTHours) - s.AdvanceDeducted) AS TotalSalary
        FROM Employees1 e
        JOIN EmployeeStat s ON e.ID = s.EmployeeID
    """)
    return cursor.fetchall()

# Add Employee

def add_employee_popup(parent, on_save_callback):
    popup = ctk.CTkToplevel(parent)
    popup.title("Add Employee")
    popup.geometry("400x500")
    popup.grab_set()

    # Title
    title_label = ctk.CTkLabel(popup, text="Add Employee", font=("Arial", 20, "bold"))
    title_label.pack(pady=10)

    # Name
    name_label = ctk.CTkLabel(popup, text="Name:")
    name_label.pack()
    name_entry = ctk.CTkEntry(popup)
    name_entry.pack(pady=5)

    # Designation
    designation_label = ctk.CTkLabel(popup, text="Designation:")
    designation_label.pack()
    designation_entry = ctk.CTkEntry(popup)
    designation_entry.pack(pady=5)

    # Basic Salary
    basic_salary_label = ctk.CTkLabel(popup, text="Basic Salary:")
    basic_salary_label.pack()
    basic_salary_entry = ctk.CTkEntry(popup)
    basic_salary_entry.pack(pady=5)

    per_day_salary_var = ctk.StringVar()

    per_day_salary_label = ctk.CTkLabel(popup, text="Per Day Salary:")
    per_day_salary_label.pack()
    per_day_salary_display = ctk.CTkEntry(popup, textvariable=per_day_salary_var)
    per_day_salary_display.configure(state="readonly")
    per_day_salary_display.pack(pady=5)

    # Calculate per-day salary as user types
    def update_per_day_salary(*args):
        try:
            basic = float(basic_salary_entry.get())
            per_day_salary = round(basic / 30, 2)
            per_day_salary_var.set(str(per_day_salary))
        except ValueError:
            per_day_salary_var.set("")

    basic_salary_entry.bind("<KeyRelease>", update_per_day_salary)

    # Status Dropdown
    status_label = ctk.CTkLabel(popup, text="Status:")
    status_label.pack()
    status_var = ctk.StringVar(value="Active")
    status_dropdown = ctk.CTkOptionMenu(popup, values=["Active", "Resigned"], variable=status_var)
    status_dropdown.pack(pady=5)

    # Joining Date (Date Picker)
    joining_label = ctk.CTkLabel(popup, text="Joining Date:")
    joining_label.pack()
    joining_entry = DateEntry(popup, width=18, background='darkblue', foreground='white', date_pattern='yyyy-mm-dd')
    joining_entry.pack(pady=5)

    # Save Button
    def save_employee():
        try:
            name = name_entry.get()
            designation = designation_entry.get()
            basic_salary = float(basic_salary_entry.get())
            per_day_salary = round(basic_salary / 30, 2)
            status = status_var.get()
            joining_date = joining_entry.get()

            # Insert into Employees1
            cursor.execute("""
                INSERT INTO Employees1 (Name, Designation, BasicSalary, Status, JoiningDate)
                VALUES (?, ?, ?, ?, ?)
            """, (name, designation, basic_salary, status, joining_date))
            conn.commit()

            # Get inserted employee ID
            cursor.execute("SELECT SCOPE_IDENTITY()")
            employee_id = cursor.fetchone()[0]

            print("Generated Employee ID:", employee_id)


            # Insert into EmployeeStats with 0 values
            cursor.execute("""
                INSERT INTO EmployeeStat (EmployeeID, OTDayAmount, OTHours, AdvanceTaken, AdvanceDeducted, TotalSalary)
                VALUES (?, 0, 0, 0, 0, 0)
            """, (employee_id,))
            conn.commit()
            def load_employees():
                cursor.execute("SELECT Name, Designation, BasicSalary, Status, JoiningDate FROM Employees1")
                rows = cursor.fetchall()
                for row in your_table_widget.get_children():
                    your_table_widget.delete(row)
                for row in rows:
                    your_table_widget.insert("", "end", values=row)


            messagebox.showinfo("Success", "Employee added successfully!")
            popup.destroy()
            on_save_callback()

        except ValueError:
            messagebox.showerror("Error", "Basic Salary must be a valid number.")
        except Exception as e:
            messagebox.showerror("Database Error", str(e))

    save_btn = ctk.CTkButton(popup, text="Save", command=save_employee)
    save_btn.pack(pady=20)

def refresh_employee_data(tree, selected_status):
    try:
        tree.delete(*tree.get_children())

        query = """
            SELECT e.ID, e.Name, e.Designation, e.BasicSalary, e.PerDaySalary, 
                   e.Status, e.JoiningDate, e.ResigningDate,
                   s.OTDayAmount, s.OTHours, s.AdvanceTaken, s.AdvanceDeducted, s.TotalSalary
            FROM Employees1 e
            JOIN EmployeeStat s ON e.ID = s.EmployeeID
        """
        if selected_status != "All":
            query += " WHERE e.Status = ?"
            cursor.execute(query, (selected_status,))
        else:
            cursor.execute(query)

        rows = cursor.fetchall()

        tree.tag_configure('evenrow', background='#f9f9f9')
        tree.tag_configure('oddrow', background='#e8e8e8')
        tree.tag_configure('Active', foreground='green')
        tree.tag_configure('resigned', foreground='red')

        for index, row in enumerate(rows):
            formatted = (
                str(row[0]), row[1], row[2],
                f"{row[3]:,.2f}", f"{row[4]:,.2f}",
                row[5], row[6], row[7],
                f"{row[8]:,.2f}", str(row[9]),
                f"{row[10]:,.2f}", f"{row[11]:,.2f}", f"{row[12]:,.2f}"
            )
            tag1 = 'evenrow' if index % 2 == 0 else 'oddrow'
            tag2 = 'Active' if row[5] == 'Active' else 'resigned'
            tree.insert('', 'end', values=formatted, tags=(tag1, tag2))

        return rows

    except Exception as e:
        messagebox.showerror("Error", str(e))
        return []


def search_employee(tree, keyword, selected_status):
    keyword = keyword.lower().strip()
    rows = refresh_employee_data(tree, selected_status)
    tree.delete(*tree.get_children())

    for index, row in enumerate(rows):
        if keyword in row[1].lower() or keyword in row[2].lower():
            formatted = (
                str(row[0]), row[1], row[2],
                f"{row[3]:,.2f}", f"{row[4]:,.2f}",
                row[5], row[6], row[7],
                f"{row[8]:,.2f}", str(row[9]),
                f"{row[10]:,.2f}", f"{row[11]:,.2f}", f"{row[12]:,.2f}"
            )
            tag1 = 'evenrow' if index % 2 == 0 else 'oddrow'
            tag2 = 'working' if row[5] == 'Working' else 'resigned'
            tree.insert('', 'end', values=formatted, tags=(tag1, tag2))


def sort_column(tv, col, reverse):
    data = [(tv.set(k, col), k) for k in tv.get_children('')]
    try:
        data.sort(key=lambda t: float(t[0].replace(",", "")), reverse=reverse)
    except:
        data.sort(reverse=reverse)
    for index, (_, k) in enumerate(data):
        tv.move(k, '', index)
    tv.heading(col, command=lambda: sort_column(tv, col, not reverse))


def open_employee_window(parent):
    emp_win = ctk.CTkToplevel(parent)
    emp_win.title("Employee Management")
    emp_win.state('zoomed')
    emp_win.focus_force()
    emp_win.grab_set()

    ctk.CTkLabel(emp_win, text="🧑‍💼 Employee Management", font=("Arial", 28, "bold")).pack(pady=10)

    # ==== Dashboard Cards ====
    card_frame = ctk.CTkFrame(emp_win)
    card_frame.pack(fill="x", padx=20, pady=10)

    def update_dashboard(frame):
        rows = refresh_employee_data(tree, status_var.get())
        total = len(rows)
        working = sum(1 for r in rows if r[5] == "Active")
        resigned = sum(1 for r in rows if r[5] == "Resigned")
        avg_salary = sum(float(r[3]) for r in rows) / total if total else 0

        metrics = [
            ("👥 Total", total, "#3498DB"),
            ("✅ Working", working, "#2ECC71"),
            ("❌ Resigned", resigned, "#E74C3C"),
            ("💰 Avg Salary", f"{avg_salary:,.2f}", "#F1C40F")
        ]
        for w in frame.winfo_children():
            w.destroy()

        for title, val, color in metrics:
            card = ctk.CTkFrame(frame, width=160, height=80, fg_color=color)
            card.pack_propagate(False)
            card.pack(side="left", padx=10)
            ctk.CTkLabel(card, text=title, font=("Arial", 11, "bold")).pack(pady=(8, 0))
            ctk.CTkLabel(card, text=str(val), font=("Arial", 20, "bold")).pack()

    # ==== Top Controls ====
    top_frame = ctk.CTkFrame(emp_win, fg_color="transparent")
    top_frame.pack(fill="x", padx=20, pady=10)

    status_var = ctk.StringVar(value="All")
    ctk.CTkLabel(top_frame, text="Status:", font=("Arial", 12)).pack(side="left", padx=5)
    status_menu = ctk.CTkOptionMenu(top_frame, variable=status_var,
                                    values=["All", "Active", "Resigned"],
                                    command=lambda _: [refresh_employee_data(tree, status_var.get()), update_dashboard(card_frame)])
    status_menu.pack(side="left", padx=10)

    ctk.CTkButton(top_frame, text="➕ Add Employee", 
        command=lambda: add_employee_popup(emp_win, lambda: [refresh_employee_data(tree, status_var.get()), update_dashboard(card_frame)])
                 ).pack(side="left", padx=10)

    search_var = ctk.StringVar()
    ctk.CTkEntry(top_frame, width=220, placeholder_text="Search by Name/Designation", textvariable=search_var).pack(side="right", padx=10)
    ctk.CTkButton(top_frame, text="🔍", command=lambda: search_employee(tree, search_var.get(), status_var.get())).pack(side="right")

    # ==== Treeview ====
    tree_frame = ctk.CTkFrame(emp_win)
    tree_frame.pack(fill="both", expand=True, padx=20, pady=10)

    columns = ["ID", "Name", "Designation", "BasicSalary", "PerDaySalary", "Status", 
               "JoiningDate", "ResigningDate", "OTDayAmount", "OTHours", 
               "AdvanceTaken", "AdvanceDeducted", "TotalSalary"]

    tree = ttk.Treeview(tree_frame, columns=columns, show="headings", height=30)
    for col in columns:
        tree.heading(col, text=col, command=lambda _col=col: sort_column(tree, _col, False))
        tree.column(col, anchor="center", width=110)

    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
    hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=tree.xview)
    tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

    vsb.pack(side="right", fill="y")
    hsb.pack(side="bottom", fill="x")
    tree.pack(fill="both", expand=True)

    refresh_employee_data(tree, status_var.get())
    update_dashboard(card_frame)


def open_orders_window():
    app.lower()
    orders_window = ctk.CTkToplevel(app)
    orders_window.title("Pending Orders")
    orders_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    orders_window.state("zoomed")
    orders_window.configure(fg_color="#0F172A")

    orders_window.lift()
    orders_window.attributes("-topmost", True)
    orders_window.focus_force()
    orders_window.after(500, lambda: orders_window.attributes("-topmost", False))
    orders_window.protocol("WM_DELETE_WINDOW", lambda: [orders_window.destroy(), app.lift()])

    ctk.CTkLabel(orders_window, text="🛒 Pending Orders", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(orders_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [orders_window.destroy(), app.lift()]).pack(pady=10)

def open_dispatch_window():
    app.lower()
    dispatch_window = ctk.CTkToplevel(app)
    dispatch_window.title("Logistics & Dispatch")
    dispatch_window.geometry(f"{app.winfo_screenwidth()}x{app.winfo_screenheight()}")
    dispatch_window.state("zoomed")
    dispatch_window.configure(fg_color="#0F172A")

    dispatch_window.lift()
    dispatch_window.attributes("-topmost", True)
    dispatch_window.focus_force()
    dispatch_window.after(500, lambda: dispatch_window.attributes("-topmost", False))
    dispatch_window.protocol("WM_DELETE_WINDOW", lambda: [dispatch_window.destroy(), app.lift()])

    ctk.CTkLabel(dispatch_window, text="🚚 Logistics & Dispatch", font=("Arial", 24, "bold"), text_color="white").pack(pady=40)
    ctk.CTkButton(dispatch_window, text="Close", text_color="white", fg_color="#1E3A8A", hover_color="#2563EB",
                  command=lambda: [dispatch_window.destroy(), app.lift()]).pack(pady=10)
    
card_info = [
    ("💰 Total Revenue", "$124,532", "+12.5%", "#2563EB", open_revenue_window),
    ("📦 Total Products", "2,847", "+8.2%", "#22C55E", open_products_window),
    ("👥 Active Employees", "1,429", "+15.3%", "#9333EA", open_employee_window),
    ("🛒 Pending Orders", "23", "-4.2%", "#F97316", open_orders_window),
    ("🚚 Logistics & Dispatch", "5", "+2.1%", "#EF4444", open_dispatch_window)
]

for title, value, change, color, func in card_info:
    card = ctk.CTkFrame(info_frame, width=200, height=100, fg_color=color, cursor="hand2")
    card.pack_propagate(False)
    card.pack(side="left", padx=10, pady=10)

    label1 = ctk.CTkLabel(card, text=title, font=("Arial", 12, "bold"))
    label1.pack(anchor="w", padx=10, pady=(5, 0))

    label2 = ctk.CTkLabel(card, text=value, font=("Arial", 24, "bold"))
    label2.pack(anchor="w", padx=10)

    change_color = "green" if "+" in change else "red"
    label3 = ctk.CTkLabel(card, text=f"{change} vs last month", font=("Arial", 10), text_color=change_color)
    label3.pack(anchor="w", padx=10)

    # Bind cards with correct function signature
    def bind_click(widget, f=func):
        try:
            if f._code_.co_argcount == 1:
                widget.bind("<Button-1>", lambda e: f(app))
            else:
                widget.bind("<Button-1>", lambda e: f())
        except AttributeError:
            pass  # Skip if it's not a callable (guard)

    for widget in [card, label1, label2, label3]:
        bind_click(widget)


# Sales Overview
bottom_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
bottom_frame.pack(fill="both", expand=True)

sales_frame = ctk.CTkFrame(bottom_frame, fg_color="#1C1F2B")
sales_frame.pack(side="left", fill="both", expand=True, padx=10, pady=10)

ctk.CTkLabel(sales_frame, text="📊 Sales Overview", font=("Arial", 16, "bold")).pack(anchor="w", padx=10, pady=10)
months = [("Jan", 45), ("Feb", 52), ("Mar", 48), ("Apr", 61), ("May", 55), ("Jun", 67)]
for m, val in months:
    ctk.CTkLabel(sales_frame, text=f"{m} - ${val}k").pack(anchor="w", padx=20)
    bar = ctk.CTkProgressBar(sales_frame, width=200)
    bar.set(val / 100)
    bar.pack(padx=20, pady=2)

ctk.CTkLabel(sales_frame, text="Total Sales: $328,000", font=("Arial", 12, "bold")).pack(anchor="w", padx=20, pady=10)

# Inventory Status
inventory_frame = ctk.CTkFrame(bottom_frame, fg_color="#1C1F2B")
inventory_frame.pack(side="left", fill="both", expand=True, padx=10, pady=10)

ctk.CTkLabel(inventory_frame, text="📦 Inventory Status", font=("Arial", 16, "bold")).pack(anchor="w", padx=10, pady=10)

inv_data = [
    ("Smartphones", 145, 200, "#3498DB"),
    ("Laptops", 89, 120, "#2ECC71"),
    ("Tablets", 67, 80, "#9B59B6"),
    ("Accessories", 234, 300, "#E67E22"),
    ("Gaming", 45, 60, "#E74C3C")
]
for name, val, maxval, clr in inv_data:
    ctk.CTkLabel(inventory_frame, text=f"{name}: {val}/{maxval}", font=("Arial", 10)).pack(anchor="w", padx=20)
    prog = ctk.CTkProgressBar(inventory_frame, width=200, progress_color=clr)
    prog.set(val / maxval)
    prog.pack(padx=20, pady=3)

total_items = sum([val for _, val, _, _ in inv_data])
ctk.CTkLabel(inventory_frame, text=f"Total Items: {total_items}", font=("Arial", 12, "bold")).pack(anchor="w", padx=20, pady=10)

# Last updated
now = datetime.datetime.now().strftime("%d/%m/%Y, %I:%M:%S %p")
ctk.CTkLabel(main_frame, text=f"Last updated: {now}", font=("Arial", 10), text_color="#888").pack(anchor="e", padx=10, pady=5)

# Your main dashboard (hidden initially)

app.withdraw()  # Hide until login is successful


import pyodbc# Login window

conn = pyodbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=SAMEER\SQLEXPRESS;"
    r"DATABASE=version1;"
    r"Trusted_Connection=yes;"
)
cursor = conn.cursor()

# 🟡 Setup Users table and dummy data
def setup_dummy_users():
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Users' AND xtype='U')
        BEGIN
            CREATE TABLE Users (
                ID INT IDENTITY(1,1) PRIMARY KEY,
                Username NVARCHAR(50) NOT NULL UNIQUE,
                Password NVARCHAR(255) NOT NULL
            );
        END
    """)
    conn.commit()

    # Insert dummy users if table is empty
    cursor.execute("SELECT COUNT(*) FROM Users")
    if cursor.fetchone()[0] == 0:
        cursor.executemany("""
            INSERT INTO Users (Username, Password) VALUES (?, ?)""", [
            ('admin', 'admin123'),
            ('Amit Sharma', 'Amit123'),
            ('Sneha Patil', 'Sneha123'),
            ('Ravi Verma', 'Ravi123'),
            ('Priya Deshmukh', 'Priya123'),
            ('Nikhil Joshi', 'Nikhil123'),
            ('Anjali Kale', 'Anjali123'),
            ('Deepak Singh', 'Deepak123')
        ])
        conn.commit()


def login_window():
    login_app = ctk.CTk()
    login_app.title("Company Login - ElectroSales")
    login_app.geometry("1200x700")
    login_app.resizable(False, False)

    ctk.CTkLabel(login_app, text="🔐 Login", font=("Arial", 24, "bold")).pack(pady=(40, 20))

    # Username
    ctk.CTkLabel(login_app, text="Username:", font=("Arial", 15)).pack(pady=(10, 0))
    username_entry = ctk.CTkEntry(login_app, placeholder_text="Enter your username")
    username_entry.pack(pady=5)

    # Password
    ctk.CTkLabel(login_app, text="Password:", font=("Arial", 15)).pack(pady=(10, 0))
    password_entry = ctk.CTkEntry(login_app, placeholder_text="Enter your password", show="*")
    password_entry.pack(pady=5)

    # Show password checkbox
    def toggle_password():
        password_entry.configure(show="" if password_entry.cget("show") == "" else "")

    ctk.CTkCheckBox(login_app, text="Show Password", command=toggle_password).pack(pady=5)

    # Validate login
    def validate_login():
        username = username_entry.get()
        password = password_entry.get()
        try:
            cursor.execute("SELECT * FROM Users WHERE Username = ? AND Password = ?", (username, password))
            user = cursor.fetchone()
            if user:
                login_app.destroy()        # First destroy login window
                app.deiconify()           # Then show the main app
                app.after(100, lambda: messagebox.showinfo("Login Successful", f"Welcome {username}!"))  # Delay to allow GUI to update
            else:
                messagebox.showerror("Invalid Login", "Incorrect username or password.")
        except Exception as e:
            messagebox.showerror("Error", str(e))


    ctk.CTkButton(login_app, text="Login", width=200, command=validate_login).pack(pady=20)
    ctk.CTkLabel(login_app, text="Username:-admin,Password:-admin123", font=("Arial", 15), text_color="#888").pack(side="bottom", pady=10)

    login_app.mainloop()

# ✅ Setup users and start login
setup_dummy_users()
login_window()

# Start main dashboard (shown only after login)
app.mainloop()
